In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
!pip install tqdm --upgrade

In [ ]:
from RoadLabsAPI.settings import credentials
import gitlab
from tqdm.notebook import tqdm
import re
from datetime import timedelta, datetime

In [ ]:
# Constants

road_labs_group_id = 1747785

In [ ]:
git = gitlab.Gitlab(
    credentials.GL_URL, private_token=credentials.GL_PRIVATE_TOKEN
)
git.auth()

## Road Labs Group

In [ ]:
# Get group

road_labs_group = git.groups.get(1747785)

In [ ]:
# Get all open issues from group

group_issues = road_labs_group.issues.list(all=True, state="opened")

In [ ]:
print("Total group open issues: {}".format(len(group_issues)))

## Projects

In [ ]:
projects = road_labs_group.projects.list(all=True)

In [ ]:
# Projects and its ids
for item in projects:
    print("{}: {}".format(item.attributes["name"], item.attributes["id"]))

In [ ]:
all_issues = []
all_mrs = []

In [ ]:
# Count issues per project

for item in projects:
    project = git.projects.get(item.attributes["id"])
    issues = project.issues.list(all=True)
    mrs = project.mergerequests.list(all=True)
    print("{}: {}".format(item.attributes["name"], len(issues)))
    all_issues += issues
    all_mrs += mrs 

In [ ]:
labels = road_labs_group.labels.list(all=True)

In [ ]:
# Count open issues per label in BOARD

board_labels = ["STT Project Backlog", "STT Sprint Backlog", "STT Doing", "STT Ready For Review",
                "STT Publ: Internal", "STT Publ: Staging", "STT Publ: Production", "STT Done"]

for name in board_labels:
    issues = road_labs_group.issues.list(all=True, state="opened", labels=[name])
    print("{}: {}".format(name, len(issues)))

## Spent time

In [ ]:
all_discussions_issues = []
all_discussions_mrs = []

In [ ]:
for issue in tqdm(all_issues):
    all_discussions_issues += issue.discussions.list()

In [ ]:
for mr in tqdm(all_mrs):
    all_discussions_mrs += mr.discussions.list()

In [ ]:
# Get issues which has spend comments

spends_issues = [a for a in all_discussions_issues if 'time spent' in a.attributes['notes'][0]['body']]
print("Total issues with spend: {}".format(len(spends_issues)))

In [ ]:
# Get MRS which has spend comments

spends_mrs = [a for a in all_discussions_mrs if 'time spent' in a.attributes['notes'][0]['body']]
print("Total MRs with spend: {}".format(len(spends_mrs)))

In [ ]:
regex = re.compile(r'((?P<days>\d+?)d)?((?P<hours>\d+?)h)?((?P<minutes>\d+?)m)?((?P<seconds>\d+?)s)?')

def parse_time(time_str):
    parts = regex.match(time_str)
    if not parts:
        return
    parts = parts.groupdict()
    time_params = {}
    for (name, param) in parts.items():
        if param:
            time_params[name] = int(param)
    return timedelta(**time_params)

def parse_discussion(discussion):
    amount = discussion.attributes['notes'][0]['body'].split(' of time')[0]
    add_or_subtract = "added" in amount
    amount = amount.strip('added ').strip('subtracted ')
    amount = parse_time(amount)
    if not add_or_subtract:
        amount = timedelta() - amount
    created_at = discussion.attributes['notes'][0]['created_at']

    return(amount, created_at)

def create_json_spends(discussions):
    json_spends = {}
    for discussion in discussions:
        project_id = discussion.attributes['project_id']
        if "issue_iid" in discussion.attributes:
            iid = discussion.attributes['issue_iid']
        elif "mr_iid" in discussion.attributes:
            iid = discussion.attributes['mr_iid']
            
        if not project_id in json_spends:
            json_spends[project_id] = {}
        if not iid in json_spends[project_id]:
            json_spends[project_id][iid] = []
        
        amount = discussion.attributes['notes'][0]['body'].split(' of time')[0]
        add_or_subtract = "added" in amount
        amount = amount.strip('added ').strip('subtracted ')
        amount = parse_time(amount)
        if not add_or_subtract:
            amount = timedelta() - amount
        created_at = datetime.strptime(discussion.attributes['notes'][0]['created_at'], '%Y-%m-%dT%H:%M:%S.%fZ')
        
        json_spends[project_id][iid].append({
            "time": amount,
            "created_at": created_at
        })
        
    return json_spends

## Milestones

In [ ]:
group_milestones = road_labs_group.milestones.list(all=True)

In [ ]:
actual_milestone = group_milestones[0]
milestone_start_date = datetime.strptime(actual_milestone.attributes["start_date"], "%Y-%m-%d")
milestone_due_date = datetime.strptime(actual_milestone.attributes["due_date"], "%Y-%m-%d")

In [ ]:
issues = road_labs_group.issues.list(all=True, milestone=actual_milestone.attributes["title"])
mrs = road_labs_group.mergerequests.list(all=True, milestone=actual_milestone.attributes["title"])

In [ ]:
# Total time already spent in actual sprint

time_spent_issues = 0
time_spent_merge_requests = 0

for issue in issues:
    time_spent_issues += issue.attributes["time_stats"]["total_time_spent"]
    
for mr in mrs:
    time_spent_merge_requests += mr.attributes["time_stats"]["total_time_spent"]

In [ ]:
# Time spent in issues before milestone

time_before_issues = timedelta(0)
json_spends_issues = create_json_spends(spends_issues)

for issue in issues:
    project_id = issue.attributes["project_id"]
    issue_iid = issue.attributes["iid"]
    if project_id in json_spends_issues:
        if issue_iid in json_spends_issues[project_id]:
            spents = json_spends_issues[project_id][issue_iid]
            
            for spent in spents:
                if spent["created_at"] < milestone_start_date:
                    time_before_issues += spent["time"]

In [ ]:
# Time spent in issues before milestone

time_before_mrs = timedelta(0)
json_spends_mrs = create_json_spends(spends_mrs)

for mr in mrs:
    project_id = mr.attributes["project_id"]
    mr_iid = mr.attributes["iid"]
    if project_id in json_spends_mrs:
        if mr_iid in json_spends_mrs[project_id]:
            spents = json_spends_mrs[project_id][mr_iid]
            
            for spent in spents:
                if spent["created_at"] < milestone_start_date:
                    time_before_mrs += spent["time"]

In [ ]:
time_before_issues = 0

In [ ]:
print("Time before in issues: \t{}".format(str(time_before_issues)))
print("Total time in issues: \t{}".format(str(timedelta(seconds=time_spent_issues))))
print("Time before in MRs: \t{}".format(str(time_before_mrs)))
print("Total time in MRs: \t{}".format(str(timedelta(seconds=time_spent_merge_requests))))